# 02_baseline_tfidf — Binario A/D

**Objetivo:** baseline clásico **robusto a ruido** (typos/transcripción) con **char TF‑IDF (3–5)** + **SVM (LinearSVC)**.  
**Justificación:** los n‑gramas de caracteres capturan patrones ortográficos aun con errores; es un buen contrapunto al enfoque rule‑based.


In [9]:
# ===============================================================
# Setup: Paths, Imports, y Utilidades Compartidas
# ===============================================================

from pathlib import Path
import pandas as pd
import re, unicodedata, os

# Intentar importar utilidades compartidas
try:
    import sys
    sys.path.insert(0, str(Path.cwd()))
    from utils_shared import setup_paths, guess_text_col, guess_label_col, normalize_label
    print("✅ Utilizando utils_shared.py")
    
    # Setup de paths centralizado
    paths = setup_paths()
    BASE_PATH = paths['BASE_PATH']
    DATA_PATH = paths['DATA_PATH']
    SPLITS_PATH = paths['SPLITS_PATH']
    
    # Usar funciones centralizadas
    _guess_text_col = guess_text_col
    _guess_label_col = guess_label_col
    _norm_label_bin = normalize_label
    
except ImportError:
    print("⚠️ utils_shared.py no encontrado, usando funciones locales")
    
    # Setup manual de paths
    BASE_PATH = Path.cwd()
    if BASE_PATH.name == "notebooks":
        BASE_PATH = BASE_PATH.parent
    
    DATA_PATH = BASE_PATH / "data"
    SPLITS_PATH = DATA_PATH / "splits"
    
    DATA_PATH.mkdir(exist_ok=True)
    
    # Funciones helper locales
    def _guess_text_col(df):
        for c in ["texto", "text", "comment", "comentario"]:
            if c in df.columns:
                return c
        return df.columns[0]
    
    def _guess_label_col(df):
        for c in ["etiqueta", "label", "category"]:
            if c in df.columns:
                return c
        return df.columns[1] if len(df.columns) > 1 else df.columns[-1]
    
    def _norm_label_bin(s):
        if pd.isna(s): 
            return ""
        s = str(s).strip().lower()
        s = unicodedata.normalize("NFKD", s).encode("ascii", "ignore").decode("ascii")
        return {'depresivo': 'depresion'}.get(s, s)

# Validar existencia de splits
if not SPLITS_PATH.exists():
    raise FileNotFoundError(
        f"❌ Splits no encontrados en {SPLITS_PATH}\n"
        f"   Debes ejecutar primero: 02_create_splits.ipynb"
    )

print(f"✅ Paths configurados:")
print(f"   BASE_PATH:   {BASE_PATH}")
print(f"   DATA_PATH:   {DATA_PATH}")
print(f"   SPLITS_PATH: {SPLITS_PATH}")

# Columnas esperadas en dataset_base.csv
TEXT_COL = "texto"
LABEL_COL = "etiqueta"

✅ Utilizando utils_shared.py
✅ Paths configurados:
   BASE_PATH:   /Users/manuelnunez/Projects/psych-phenotyping-paraguay
   DATA_PATH:   /Users/manuelnunez/Projects/psych-phenotyping-paraguay/data
   SPLITS_PATH: /Users/manuelnunez/Projects/psych-phenotyping-paraguay/data/splits


## 1) Carga y preprocesamiento **agresivo** (pensado para ML clásico)

In [10]:
# ===============================================================
# Carga de Datos y Preprocesamiento AGRESIVO (TF-IDF + SVM)
# ===============================================================
#
# ESTRATEGIA DE PREPROCESAMIENTO: AGRESIVA (máxima normalización)
#
# ¿Por qué preprocesamiento agresivo?
#
# 1. **Convierte a minúsculas**:
#    - "Depresión" y "depresión" son la misma palabra para el modelo
#    - TF-IDF trata cada variación como token diferente sin lowercase
#    - Reduce vocabulario (~40%) mejorando generalización
#
# 2. **Elimina acentos/tildes**:
#    - "depresión" → "depresion"
#    - Robustez ante errores de transcripción (común en notas clínicas)
#    - Los datos tienen muchos typos: "anciedad", "deprecion", etc.
#
# 3. **Elimina símbolos especiales**:
#    - Preserva solo: letras, números, espacios, puntuación básica
#    - Elimina ruido de transcripción: emojis, símbolos raros, etc.
#
# 4. **Colapsa alargamientos**:
#    - "holaaaaa" → "holaa" (igual que rule-based)
#    - Mantiene énfasis sin explotar el vocabulario
#
# 5. **Marca negaciones** (innovación clave):
#    - "no tengo apetito" → "no_tengo apetito"
#    - Permite a TF-IDF capturar "no_X" como feature diferente de "X"
#    - Proxy simple para manejar negación sin ConText
#
# ¿Qué NO hace este preprocesamiento?
# - ❌ No hace stemming/lemmatización (puede romper char n-grams)
# - ❌ No elimina stopwords (útiles en contexto clínico)
# - ❌ No tokeniza (char-level TF-IDF lo hace automáticamente)
#
# Comparación con otros baselines:
# - Rule-Based: Conservador (preserva tildes/mayúsculas para patterns)
# - TF-IDF: Agresivo (máxima normalización para robustez)
# - BETO: Mínimo (solo tokenización, el modelo maneja el resto)
#
# Justificación del char TF-IDF (3-5):
# - **Robusto a typos**: "deprecion" captura "pre", "rec", "epr", "ecion" → overlap con "depresion"
# - **No requiere tokenización perfecta**: funciona character-by-character
# - **Captura patrones morfológicos**: sufijos -ción, -dad, -oso, etc.
# - **Estándar de la industria** para texto con ruido (OCR, transcripción)
#
# ===============================================================

import pandas as pd, re, unicodedata

# Cargar splits unificados desde 02_create_splits.ipynb
dataset_base = pd.read_csv(SPLITS_PATH / 'dataset_base.csv')
train_indices = pd.read_csv(SPLITS_PATH / 'train_indices.csv')['row_id'].values
val_indices = pd.read_csv(SPLITS_PATH / 'val_indices.csv')['row_id'].values

print(f"✅ Splits cargados desde {SPLITS_PATH}/:")
print(f"   Train: {len(train_indices)} ejemplos")
print(f"   Val:   {len(val_indices)} ejemplos")
print(f"   Total: {len(dataset_base)} ejemplos en dataset_base.csv")

# Detectar columnas automáticamente
text_col = _guess_text_col(dataset_base)
label_col = _guess_label_col(dataset_base)
print(f"\n📋 Columnas detectadas: texto='{text_col}', label='{label_col}'")

# Definir función de limpieza agresiva
RE_MULTI = re.compile(r'(.)\1{2,}')  # Detecta 3+ letras repetidas

def clean_text_ml(s: str) -> str:
    """
    Limpieza AGRESIVA para TF-IDF (máxima normalización para robustez).
    
    Aplica:
    - Lowercase completo
    - Normalización NFC + eliminación de tildes
    - Colapso de alargamientos
    - Eliminación de símbolos especiales (preserva puntuación básica)
    - Marca negaciones simples ("no X" → "no_X")
    """
    if pd.isna(s):
        return ""
    
    s = str(s).lower().strip()           # Lowercase total
    s = unicodedata.normalize("NFC", s)  # Normaliza tildes
    s = RE_MULTI.sub(r'\1\1', s)         # Colapsa alargamientos
    
    # Elimina símbolos especiales (preserva letras, números, puntuación básica)
    s = re.sub(r"[^a-z0-9áéíóúüñ\s.,!?:/\-]", " ", s)
    s = re.sub(r"\s+", " ", s).strip()   # Normaliza espacios
    
    # Marca negaciones simples: "no tengo" → "no_tengo"
    s = re.sub(r"\bno\s+([a-záéíóúüñ]{2,})", r"no_\1", s)
    
    return s

# Aplicar limpieza agresiva
dataset_base['texto_ml'] = dataset_base[text_col].map(clean_text_ml)

print(f"\n🧹 Aplicado preprocesamiento agresivo (lowercase + sin tildes + negaciones)")
print(f"   Ejemplo antes: {dataset_base[text_col].iloc[0][:80]}...")
print(f"   Ejemplo después: {dataset_base['texto_ml'].iloc[0][:80]}...")

# Separar train y val usando índices guardados
df_train = dataset_base[dataset_base['row_id'].isin(train_indices)].copy()
df_val = dataset_base[dataset_base['row_id'].isin(val_indices)].copy()

X_train, y_train = df_train['texto_ml'], df_train[label_col]
X_val, y_val = df_val['texto_ml'], df_val[label_col]

print(f"\n📊 Splits creados:")
print(f"\nTrain ({len(df_train)} ejemplos):")
train_dist = y_train.value_counts()
for label, count in train_dist.items():
    print(f"   {label}: {count}")

print(f"\nVal ({len(df_val)} ejemplos):")
val_dist = y_val.value_counts()
for label, count in val_dist.items():
    print(f"   {label}: {count}")

✅ Splits cargados desde /Users/manuelnunez/Projects/psych-phenotyping-paraguay/data/splits/:
   Train: 2500 ejemplos
   Val:   626 ejemplos
   Total: 3126 ejemplos en dataset_base.csv

📋 Columnas detectadas: texto='texto', label='etiqueta'

🧹 Aplicado preprocesamiento agresivo (lowercase + sin tildes + negaciones)
   Ejemplo antes: Reposicion de medicacion 2) EXAMEN FISICO GRAL. Y GINECOLOGICO PESO ( ) TALLA ( ...
   Ejemplo después: reposicion de medicacion 2 examen fisico gral. y ginecologico peso talla presion...

📊 Splits creados:

Train (2500 ejemplos):
   depresion: 1760
   ansiedad: 740

Val (626 ejemplos):
   depresion: 441
   ansiedad: 185

🧹 Aplicado preprocesamiento agresivo (lowercase + sin tildes + negaciones)
   Ejemplo antes: Reposicion de medicacion 2) EXAMEN FISICO GRAL. Y GINECOLOGICO PESO ( ) TALLA ( ...
   Ejemplo después: reposicion de medicacion 2 examen fisico gral. y ginecologico peso talla presion...

📊 Splits creados:

Train (2500 ejemplos):
   depresion: 1760

## 2) Split estratificado y entrenamiento (char TF‑IDF + LinearSVC)

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC

tfidf_char = TfidfVectorizer(
    analyzer='char_wb',
    ngram_range=(3,5),
    min_df=2,
    max_df=0.95
)

clf = Pipeline([
    ('tfidf', tfidf_char),
    ('svm', LinearSVC(class_weight='balanced', random_state=42))
])

clf.fit(X_train, y_train)
y_pred = clf.predict(X_val)
print("✅ Entrenamiento completado.")

✅ Entrenamiento completado.


## 3) Métricas y exportables

In [12]:
from sklearn.metrics import classification_report, f1_score, precision_score, recall_score, confusion_matrix

tfidf_pred_csv   = DATA_PATH/'tfidf_predictions.csv'
tfidf_report_csv = DATA_PATH/'tfidf_classification_report.csv'
tfidf_eval_csv   = DATA_PATH/'tfidf_eval.csv'
tfidf_cm_csv     = DATA_PATH/'tfidf_confusion_matrix.csv'

classes = ['depresion','ansiedad']

pd.DataFrame(classification_report(y_val, y_pred, labels=classes, output_dict=True, zero_division=0))  .transpose().to_csv(tfidf_report_csv, index=True, encoding='utf-8')

pd.DataFrame([{
    'macro_f1': f1_score(y_val, y_pred, average='macro', zero_division=0),
    'macro_precision': precision_score(y_val, y_pred, average='macro', zero_division=0),
    'macro_recall': recall_score(y_val, y_pred, average='macro', zero_division=0),
    'n': int(len(y_val))
}]).to_csv(tfidf_eval_csv, index=False, encoding='utf-8')

cm = confusion_matrix(y_val, y_pred, labels=classes)
pd.DataFrame(cm, index=[f'true_{c}' for c in classes], columns=[f'pred_{c}' for c in classes]).to_csv(tfidf_cm_csv)

pd.DataFrame({'texto': X_val, 'y_true': y_val, 'y_pred': y_pred}).to_csv(tfidf_pred_csv, index=False, encoding='utf-8')

print("✅ Exportados:")
print(" - Predicciones:", tfidf_pred_csv)
print(" - Reporte:", tfidf_report_csv)
print(" - Métricas:", tfidf_eval_csv)
print(" - Matriz:", tfidf_cm_csv)

✅ Exportados:
 - Predicciones: /Users/manuelnunez/Projects/psych-phenotyping-paraguay/data/tfidf_predictions.csv
 - Reporte: /Users/manuelnunez/Projects/psych-phenotyping-paraguay/data/tfidf_classification_report.csv
 - Métricas: /Users/manuelnunez/Projects/psych-phenotyping-paraguay/data/tfidf_eval.csv
 - Matriz: /Users/manuelnunez/Projects/psych-phenotyping-paraguay/data/tfidf_confusion_matrix.csv
